In [ ]:
from pyspark.sql.functions import expr, datediff, col, to_date

In [ ]:
# Create Spark session
spark = SparkSession.builder.appName("DeltaToSQL").getOrCreate()

#  Set up Azure Blob Storage access
spark.conf.set(
    "fs.azure.account.key.lhindstorage.blob.core.windows.net",
    "SPFV+Hdv7AWTGTpysBBwEVXJvefoYSSr17wLSWB+onc3PMYbwXEcpFRZvHvXF06eePtCy00PIvZu+AStpz1VzA=="
)

In [ ]:
#  Load clean CSV datasets from Azure Blob
base_path = "wasbs://clean-data@lhindstorage.blob.core.windows.net"

orders = spark.read.option("header", "true").csv(f"{base_path}/orders.csv")
order_items = spark.read.option("header", "true").csv(f"{base_path}/order_items.csv")
customers = spark.read.option("header", "true").csv(f"{base_path}/customers.csv")
products = spark.read.option("header", "true").csv(f"{base_path}/products.csv")
sellers = spark.read.option("header", "true").csv(f"{base_path}/sellers.csv")
geolocation = spark.read.option("header", "true").csv(f"{base_path}/geolocation.csv")

In [ ]:
# Create dim tables
dim_products = products.dropDuplicates(["product_id"])

dim_sellers = sellers.join(
    geolocation,
    sellers["seller_zip_code_prefix"] == geolocation["geolocation_zip_code_prefix"],
    "left"
).drop("geolocation_zip_code_prefix")

dim_customers = customers.join(
    geolocation,
    customers["customer_zip_code_prefix"] == geolocation["geolocation_zip_code_prefix"],
    "left"
).drop("geolocation_zip_code_prefix")

dim_dates = orders.select(
    to_date("order_purchase_timestamp").alias("purchase_date"),
    to_date("order_delivered_customer_date").alias("delivery_date")
).dropDuplicates()

In [ ]:
# Create fact table
fact_order_items = order_items.join(orders, "order_id", "left")
    ))

In [ ]:
jdbc_url = (
    "jdbc:sqlserver://server-lhind.database.windows.net:1433;"
    "database=db-lhind-dev;"
    "user=lufthansadbo@server-lhind;"
    "password=Lhind2025!;"
    "encrypt=true;"
    "trustServerCertificate=false;"
    "hostNameInCertificate=*.database.windows.net;"
    "loginTimeout=30;"
)


In [ ]:
fact_order_items.write.jdbc(url=jdbc_url, table="fact_order_items", mode="overwrite", properties=connection_properties)
dim_customers.write.jdbc(url=jdbc_url, table="dim_customers", mode="overwrite", properties=connection_properties)
dim_products.write.jdbc(url=jdbc_url, table="dim_products", mode="overwrite", properties=connection_properties)
dim_sellers.write.jdbc(url=jdbc_url, table="dim_sellers", mode="overwrite", properties=connection_properties)
dim_dates.write.jdbc(url=jdbc_url, table="dim_dates", mode="overwrite", properties=connection_properties)
